In [1]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv

## Load EBOM, Chi's Supply Chain BOM and Shaolong's BOM

Engineering BOM (EBOM). Updated weekly and has the most up-to-date Part Number and Revision. Supply Chain BOM is based on EBOM with additional sourcing and supplier quality information.

In [2]:
EBOM = pd.read_excel("/home/yifei/Documents/BOM_tool/EBOM 8.23.18.xlsx")
# new_header = EBOM.iloc[0]
# EBOM = EBOM[1:]
# EBOM.columns = new_header
EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
print("Engineering VP BOM Shape: ",EBOM.shape)
EBOM.head(5)
# print(EBOM.columns.values)
# EBOM.shape





Engineering VP BOM Shape:  (3164, 14)


,Identifier,Title,Revision,Description,QTY,UOM,Purchased Part Type,Maturity,Part Type,System,SubSystem,Legacy Part Number,Legacy Part Revision,Configuration
0,0,AA-ZZ000001NN-03,A.1,PHTM EC001 VP,1,EA EACH,NaN,In Work,Phantom,Z Vehicle Top Level Cfg,Z Z NA,NaN,NaN,NaN
1,1,AA-AZ000002NN-03,A.1,PHTM BIW VP,1,EA EACH,NaN,In Work,Phantom,A BIW,A Z NA,NaN,NaN,NaN
2,1|1,AA-AF000002NN-03,A.1,PHTM BIW ROOF STRUCTURE VP,1,EA EACH,NaN,In Work,Phantom,A BIW,A F Roof Structure,NaN,NaN,NaN
3,1|1|1,AA-AF003414NN-00,A.1,FULL_ROOF,1,EA EACH,NaN,In Work,NaN,A BIW,A F Roof Structure,EC001_02_521_000,NaN,NaN
4,1|1|1|1,AA-AF003415NN-00,A.1,FULL ROOF,1,NaN,NaN,In Work,NaN,A BIW,A F Roof Structure,EC001_02_501_101_PRT_019,19,NaN


In [3]:
print(EBOM.columns.values)

['Identifier' 'Title' 'Revision' 'Description' 'QTY' 'UOM'
 'Purchased Part Type' 'Maturity' 'Part Type' 'System' 'SubSystem'
 'Legacy Part Number' 'Legacy Part Revision' 'Configuration']


Removing Phantom part and Fasterners and filter only "Buy" part leaves us 1461 Unique PNs

In [4]:
EBOM_filtered = EBOM[(EBOM["Purchased Part Type"]=="Buy") & (EBOM["Part Type"]!="Phantom") & (EBOM["System"]!="Y Fasteners")]
print(EBOM_filtered.shape)


(1461, 14)


Chi's SCBOM. We need the column names from Chi's SCBOM

In [5]:
SCBOM_chi = pd.read_excel("/home/yifei/Documents/BOM_tool/SCBOM.xlsx")
print("Supply Chain BOM Shape: ",SCBOM_chi.shape)
SCBOM_chi.head(20)


Supply Chain BOM Shape:  (6253, 47)


,Key,Identifier,PartNumber,PartRevision,LegacyPartNumber,LegacyPartRevision,System,SubSystem,Description,QTY,...,LastPickupDate,SC_PackagingeProposalApproved,PartStatus,PartActive,Maturity,RevisionComment,Configuration,EngineeringReleaseEstimate,EngineeringReleaseActual,AssemblyStation
0,AA-YD004238NN-00A.1EC001_04_999_0059VP,NaN,AA-YD004238NN-00,A.1,EC001_04_999_0059,NaN,EXTERIOR -STANDARD PARTS,REAR BUMPER,FLANGE NUT M6,6.0,...,NaT,False,Removed,Inactive,NaN,Set2 Count = 1 Set3 Count = 1,NaN,NaN,NaN,NaN
1,(2)7052134730A.1VP,11|3|8|1|8,(2)7052134730,A.1,NaN,NaN,NaN,NaN,NaN,6.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
2,(2)7282-6443-40_Dup4A.1VP,11|3|8|1|18,(2)7282-6443-40_Dup4,A.1,NaN,NaN,NaN,NaN,NaN,3.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
3,(2)7283-3441-40_prd-BYT-00606114A.1VP,11|3|17|2|5,(2)7283-3441-40_prd-BYT-00606114,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
4,_DMU7282-6453-60A.1VP,11|3|17|2|10,_DMU7282-6453-60,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
5,_DMU7283645440.1A.1VP,11|3|17|2|7,_DMU7283645440.1,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
6,_FLT2471952XA.1VP,4|4|7|1,_FLT2471952X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
7,_FLT2471954XA.1VP,4|4|8|1,_FLT2471954X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
8,_FLT2471955XA.1VP,4|4|9|1,_FLT2471955X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
9,_FLT2492062XA.1VP,4|4|3|1,_FLT2492062X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN


In [6]:
SCBOM_chi.columns

Index(['Key', 'Identifier', 'PartNumber', 'PartRevision', 'LegacyPartNumber',
       'LegacyPartRevision', 'System', 'SubSystem', 'Description', 'QTY',
       'Type', 'Engineer', 'Program', 'PurchasePartType', 'NominatedSupplier',
       'ConfirmedSupplier', 'GSM', 'RFQSent', 'RFQReceived', 'LOIReceived',
       'SourcingCouncilScheduledDate', 'SourcingCouncilAprovedDate', 'DSA',
       'GTC', 'AwardLetter', 'PurchaseRequistion', 'PurchaseOrder',
       'ManufacturingLT', 'EngineeringReleaseNeededBy', 'CreatedDate',
       'GSMComment', 'ModifiedDate', 'FirstPickupDate', 'CCCRequired',
       'PieceCost', 'CustomComplianceCompleted', 'SubCommodity',
       'LastPickupDate', 'SC_PackagingeProposalApproved', 'PartStatus',
       'PartActive', 'Maturity', 'RevisionComment', 'Configuration',
       'EngineeringReleaseEstimate', 'EngineeringReleaseActual',
       'AssemblyStation'],
      dtype='object')

In [7]:
PreAP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="PAP"]
print("Pre AP BOM shape: ", PreAP_BOM_chi.shape)
AP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="AP"]
print("AP BOM shape: ", AP_BOM_chi.shape)
VP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="VP"]
print("VP BOM shape: ", VP_BOM_chi.shape)

#Active VP BOM with latest part revision
VP_BOM_Activate_chi = VP_BOM_chi[(VP_BOM_chi["PartActive"]=="Active")]
print("VP Active BOM: ", VP_BOM_Activate_chi.shape)

VPM_BOM_Activate_chi_removePhantomPart = VP_BOM_Activate_chi[VP_BOM_Activate_chi["Type"]!="Phantom"]
print("removed phatom", VPM_BOM_Activate_chi_removePhantomPart.shape)

                                                   


Pre AP BOM shape:  (863, 47)
AP BOM shape:  (0, 47)
VP BOM shape:  (5390, 47)
VP Active BOM:  (5064, 47)
removed phatom (5000, 47)


Load shaolong's SC BOM which has the most sourcing information updated by buyers and SQEs. 

In [8]:
SCBOM_shaolong = pd.read_excel("/home/yifei/Documents/BOM_tool/Shaolong BOM.xlsx", index_col=None)
# SCBOM_shaolong.reset_index()
SCBOM_shaolong

,No.,SYSTEM,Commodity Package,Count Commodity,Description,BYTON P/N,ENGINEER,NJ IPM,GSM,Senior GSM,...,Hard tool Completion,Home line,FTN,Grain,AAR,Run at rate,PV Completion,Phase 1 PSW,Phase 3 PSW,Comments
0,1900-01-01,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,1.0,12V POWER OUTLET,AA-LE000742NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
1,1900-01-02,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,0.0,CIGAR LIGHTER SOCKET,AA-LE001138NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
2,1900-01-03,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,0.0,CIGAR LIGHTER POPOUT,AA-LE001137NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
3,1900-01-04,LOW VOLTAGE SYSTEMS (E/E),12V BATTERY,1.0,12V BATTERY,AA-LA000707NN-00,rockymark.juan@byton.com,NaN,Jennie.Lu,Veronica.Park,...,,,,,,,,,,"supplier should be ""C&D"""
4,1900-01-05,LOW VOLTAGE SYSTEMS (E/E),230 V outlet,1.0,230V HIGH VOLTAGE POWER OUTLET,AA-LE000746NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
5,1900-01-06,INTELLIGENT CAR ENTERTAINMENT (ICE),4 ARRAY MEMS MICROPHONE,1.0,4 ARRAY MEMS MICROPHONE,AA-ND000703FN-00,CHET.DIKEN@BYTON,Ryan.Dong,Ryan.Dong,Veronica.Park,...,,,,,,,,,,"Byton Inhouse design, production supplier not ..."
6,1900-01-07,CHASSIS,ACCELERATOR PEDAL,1.0,ACCELERATOR PEDAL LHD ASSY,AA-EB110100FN-00,gary.waite@ricardo.com,NaN,Tom.Peleg,Tom.Peleg,...,,,,,,,,,,Parts already SOP for Other customer
7,1900-01-08,INTELLIGENT CAR ENTERTAINMENT (ICE),ACOUSTIC VEHICLE ALERT SYSTEM,1.0,ACOUSTIC VEHICLE ALERT SYSTEM,AA-NH003864FN-03,MARKUS.KOCH@BYTON.COM,NaN,Veronica.Park,Veronica.Park,...,,,,,,,,,,Not sourced for SOP
8,1900-01-09,INTERIOR,ACU,1.0,AIRBAG CONTROL UNIT,AA-DJ000855NN-00,JOHN.SCHEICK@BYTON.COM,Kun Fang,Kun Fang,Shaolong.Huang,...,,,,,,,,,,No feedback form supplier
9,1900-01-10,EXTERIOR,AERO PNL D-PILLAR,1.0,ASSY AERO PNL D-PILLAR LH,AA-CF003398NL-00,Mark Bruderick,Mark Shen,Mark Shen,Harvey.Li,...,,,,,,,,,,


In [9]:
SCBOM_shaolong.columns.values

array(['No.', 'SYSTEM', 'Commodity Package', 'Count Commodity',
       'Description', 'BYTON P/N', 'ENGINEER', 'NJ IPM', 'GSM',
       'Senior GSM', 'VP\nSUPPLIER\n(Name)', 'Supplier Sourced?',
       'Supplier \nContact Person', 'Supplier \nContact E-mail',
       'Supplier Location', 'Over Sea Plant?',
       'FOB / FAC &Where?\n(if over sea)', 'Tier', 'Ship to',
       'GTC & Contract', 'SQE Priority?', 'Risky Esclation', 'JIS?',
       'Oversea Pick Up Location', 'Package cost confirmed?',
       'Transport Price quoted?', 'Daily Production Capability',
       'Order Leader Time', 'Mnimum Order Quantity', 'SQE', 'Priority',
       'Reason for Priority', 'APQP \nKICK OFF', 'Unnamed: 33',
       'Tooling\nLead time', 'Tool Kick off Plan', 'Hard tool Completion',
       'Home line', 'FTN', 'Grain', 'AAR', 'Run at rate', 'PV Completion',
       'Phase 1 PSW', 'Phase 3 PSW', 'Comments'], dtype=object)

In [10]:
print("Supply Chain BOM Shape: ",SCBOM_shaolong.shape)

Supply Chain BOM Shape:  (1516, 46)


## Step 0: Create a blank Supply Chain BOM using EBOM

In [11]:
# copy EBOM to the new SCBOM
print("EBOM shape is: ", EBOM.shape)
SCBOM = EBOM.copy()
print("SCBOM shape is: ", SCBOM.shape)
#specify SCBOM columns
SCBOM.columns.values

EBOM shape is:  (3164, 14)
SCBOM shape is:  (3164, 14)


array(['Identifier', 'Title', 'Revision', 'Description', 'QTY', 'UOM',
       'Purchased Part Type', 'Maturity', 'Part Type', 'System',
       'SubSystem', 'Legacy Part Number', 'Legacy Part Revision',
       'Configuration'], dtype=object)

In [12]:
ebom_column_header = EBOM.columns.values

sourcing_column_header = ["Supplier Name", "Supplier Contact Person", "Supplier Contact Email",
                          "Supplier Manufacturing Location", "GSM", "GSM Manager", "Engineer", 
                          "Engineer Manager", "RFQ Sent", "RFQ Received", "LOI Received", 
                          "Sourcing Council Scheduled Date", "SourcingCouncilAprovedDate", 
                          "DSA", "GTC", "Award Letter", "Purchae Requisition", "Purchase Order",
                          "Sourcing Status",]

logistics_column_header = ["First Pickup Date", "Last Pickup Date", "Packaging Proposal Approved"]

sqe_column_header = ["SQE", "Priority", "APQP Kick Off", "APQP Status","Tooling Lead Time", 
                     "Tooling Kick Off Date", "Tooling Completion Date", "Home Line", "FTN", 
                     "Grain", "AAR", "Run At Rate", "PV Completion Date", "Phase 1 PSW Date", 
                     "Phase 3 PSW Date", "Comments"]

log_column_header = ["Part Creation Date", "Last Modified Date", "Part Status", "Part Active"]

column_header = sourcing_column_header + logistics_column_header + sqe_column_header + log_column_header

In [13]:
# add additional columns
def add_columns(df, columns):
    df2 = pd.DataFrame(columns=columns)
    df = df.append(df2, sort=False, verify_integrity=True)
    return df

In [14]:
SCBOM = add_columns(SCBOM, column_header)
SCBOM.shape

(3164, 56)

## Step 1: Update new SCBOM using Shaolong's BOM

In [15]:
# # search PN and Rev in shaolong's SCBOM
# def search(PN, Rev):
#     for index, row in SCBOM_shaolong.iterrows():
#         if (row["BYTON P/N"] == PN and row["BYTON REVISION"]==Rev):
#             return index  
#         else:
#             continue;
#         return index

In [16]:
# print(search("AA-YV004051NN-00", "A.1"))

In [17]:
df = pd.DataFrame({'BoolCol': [True, False, False, True, True], 'Name': [1,2,3,4,5]},
       index=[10,20,30,40,50])
idx = df.index[df['BoolCol']]
idx

Int64Index([10, 40, 50], dtype='int64')

In [18]:
df.loc[idx[0]]["Name"]

1

In [19]:
df

,BoolCol,Name
10,True,1
20,False,2
30,False,3
40,True,4
50,True,5


In [20]:
df.loc[(df["Name"]==2) & (df["BoolCol"]==False)].index.values

array([20])

In [21]:
df.loc[[10], ["Name"]] = df.loc[[20], 'Name'].values
df

,BoolCol,Name
10,True,2
20,False,2
30,False,3
40,True,4
50,True,5


In [34]:
# save dataframe to Excel
def save(df):
    #https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
    df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

    writer = pd.ExcelWriter('Supply Chain BOM.xlsx')
    df.to_excel(writer, sheet_name="Supply Chain BOM" )
    writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [23]:
# search PN and Rev in shaolong's SCBOM
def search(PN):
    # shaolong's data doesn't have Rev.
#     df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN) & (SCBOM_shaolong["BYTON REVISION"]==Rev)]
    df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN)]
    index = df.index
    if (len(index)==1):
        return index
    else:
        return index[0:1]
        

In [24]:
print(search("AA-AF003416NN-00"))

Int64Index([261], dtype='int64')


In [25]:
print(search("AA-YF000148NN-00"))

Int64Index([1105], dtype='int64')


In [26]:
column_header

['Supplier Name',
 'Supplier Contact Person',
 'Supplier Contact Email',
 'Supplier Manufacturing Location',
 'GSM',
 'GSM Manager',
 'Engineer',
 'Engineer Manager',
 'RFQ Sent',
 'RFQ Received',
 'LOI Received',
 'Sourcing Council Scheduled Date',
 'SourcingCouncilAprovedDate',
 'DSA',
 'GTC',
 'Award Letter',
 'Purchae Requisition',
 'Purchase Order',
 'Sourcing Status',
 'First Pickup Date',
 'Last Pickup Date',
 'Packaging Proposal Approved',
 'SQE',
 'Priority',
 'APQP Kick Off',
 'APQP Status',
 'Tooling Lead Time',
 'Tooling Kick Off Date',
 'Tooling Completion Date',
 'Home Line',
 'FTN',
 'Grain',
 'AAR',
 'Run At Rate',
 'PV Completion Date',
 'Phase 1 PSW Date',
 'Phase 3 PSW Date',
 'Comments',
 'Part Creation Date',
 'Last Modified Date',
 'Part Status',
 'Part Active']

In [27]:
list(SCBOM_shaolong.columns.values)

['No.',
 'SYSTEM',
 'Commodity Package',
 'Count Commodity',
 'Description',
 'BYTON P/N',
 'ENGINEER',
 'NJ IPM',
 'GSM',
 'Senior GSM',
 'VP\nSUPPLIER\n(Name)',
 'Supplier Sourced?',
 'Supplier \nContact Person',
 'Supplier \nContact E-mail',
 'Supplier Location',
 'Over Sea Plant?',
 'FOB / FAC &Where?\n(if over sea)',
 'Tier',
 'Ship to',
 'GTC & Contract',
 'SQE Priority?',
 'Risky Esclation',
 'JIS?',
 'Oversea Pick Up Location',
 'Package cost confirmed?',
 'Transport Price quoted?',
 'Daily Production Capability',
 'Order Leader Time',
 'Mnimum Order Quantity',
 'SQE',
 'Priority',
 'Reason for Priority',
 'APQP \nKICK OFF',
 'Unnamed: 33',
 'Tooling\nLead time',
 'Tool Kick off Plan',
 'Hard tool Completion',
 'Home line',
 'FTN',
 'Grain',
 'AAR',
 'Run at rate',
 'PV Completion',
 'Phase 1 PSW',
 'Phase 3 PSW',
 'Comments']

In [42]:
# copy Shaolong's BOM info and paste into SCBOM
def copy_and_paste_row(df1, index1, df2, index2):
#     data = str(data)[1:-1]
    df1.loc[index1,["GSM"]] = str(df2.loc[index2,["GSM"]].values).replace('[', '').replace(']', '').replace('\'', '')
    df1.loc[index1,["GSM Manager"]] = str(df2.loc[index2,["Senior GSM"]].values).replace('[', '').replace(']', '').replace('\'', '')
    df1.loc[index1,["Engineer"]] = str(df2.loc[index2,["ENGINEER"]].values).replace('[', '').replace(']', '').replace('\'', '')
     df1.loc[index1,["Engineer"]] = str(df2.loc[index2,["ENGINEER"]].values).replace('[', '').replace(']', '').replace('\'', '')
        df1.loc[index1,["Engineer"]] = str(df2.loc[index2,["ENGINEER"]].values).replace('[', '').replace(']', '').replace('\'', '')
             df1.loc[index1,["Engineer"]] = str(df2.loc[index2,["ENGINEER"]].values).replace('[', '').replace(']', '').replace('\'', '')
                 df1.loc[index1,["Engineer"]] = str(df2.loc[index2,["ENGINEER"]].values).replace('[', '').replace(']', '').replace('\'', '') df1.loc[index1,["Engineer"]] = str(df2.loc[index2,["ENGINEER"]].values).replace('[', '').replace(']', '').replace('\'', '')
    
    return df1

IndentationError: unexpected indent (<ipython-input-42-6d3cda9a1027>, line 7)

In [29]:
print(str(SCBOM_shaolong.loc[1000, ["GSM"]].values).replace('[', '').replace(']', '').replace('\'', ''))

Joseph Zhu


In [37]:
# update SCBOM with inputs from Shaolong's BOM
for index, row in SCBOM.iterrows():
    #loop through EBOM by PN
    PN = row["Title"]
    Rev = row["Revision"]
    #find out if this PN and Rev exist in Shaolong's BOM, and return the index
    index_shaolong = search(PN)
    #if not found
    if (index_shaolong.values == []):
        #do nothing
        continue
    #if PN found, update SCBOM with Shaolong's BOM info
    else:
        #copy shaolong's BOM info to SCBOM
        copy_and_paste_row(SCBOM, index, SCBOM_shaolong, index_shaolong)


        
            
        
        
    
    

/home/yifei/miniconda3/envs/BOM/lib/python3.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [31]:
SCBOM.loc[[21]]

,Identifier,Title,Revision,Description,QTY,UOM,Purchased Part Type,Maturity,Part Type,System,...,AAR,Run At Rate,PV Completion Date,Phase 1 PSW Date,Phase 3 PSW Date,Comments,Part Creation Date,Last Modified Date,Part Status,Part Active
21,1|2|1|1|1|4|1,AA-YF000148NN-00,A.1,HEXAGONAL RIVET NUT M6,1.0,EA EACH,Buy,Released,Part,Y Fasteners,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
save(SCBOM)

## Create a new jupyter notebook for this step
### Step 2: Maintain and update SCBOM using updated EBOM